In [7]:
%pip install faiss-cpu sentence-transformers ollama tika sentencepiece PyPDF2 duckduckgo_search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.2 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import nltk
nltk.download('punkt')

import PyPDF2

def pdf_to_text(file_path):
    
    with open(file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    
def to_articles(texts):
    articles = [article for article in texts.split("\n") if len(article) > 20]
    return articles

# Подготовка данных
books = ['1.pdf', '2.pdf', '3.pdf', '4.pdf', '5.pdf']
documents = []
for filename in books:
    documents.append(pdf_to_text(filename))


[nltk_data] Downloading package punkt to /home/lockr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
incorrect startxref pointer(1)
incorrect startxref pointer(1)
incorrect startxref pointer(1)


In [11]:
from pymystem3 import Mystem
import re
import string
import nltk
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
tqdm.pandas()
nltk.download('stopwords')
def standartize_text(df):
    m = Mystem()
    sw = stopwords.words("russian")
    df = df.replace(to_replace=r'[^\w\s]', value=" ", regex=True)
    df = df.replace(to_replace=r"(.)\1\1+", value=r'\1\1', regex=True)
    list_patterns = [r'\b[^\W\dА-Яа-яёЁ]+\b', f'[{string.punctuation}]', '/\r?\n|\r/', '\n']
    pattern = re.compile("|".join(list_patterns))
    df = df.replace(to_replace=pattern, value='', regex=True)
    df = df.replace(to_replace=r'\s+', value=' ', regex=True).str.strip()
    df = df.progress_apply(lambda x: " ".join([word for word in m.lemmatize(x) if word not in sw]))
    df = df.str.strip()
    return df

raw_documents = []
for doc in documents:
    raw_documents.extend(to_articles(doc))

raw_documents = pd.DataFrame({'text': raw_documents})

clean = standartize_text(raw_documents['text'])


[nltk_data] Downloading package stopwords to /home/lockr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/3916 [00:00<?, ?it/s]

In [11]:
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Используем SentenceTransformer для создания векторов
embedder = SentenceTransformer('all-MiniLM-L6-v2', tokenizer_kwargs={'clean_up_tokenization_spaces':True})
documents = clean
doc_embeddings = embedder.encode(documents)

# Создание и добавление в индекс
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))


In [22]:
from ollama import Client

# Инициализируй модель OLLAMA
model = Client(
    host="http://localhost:11434"
)

def retrieve_and_generate(query, top_k=3):
    # Поиск релевантных документов
    query_embedding = embedder.encode([query])
    _, I = index.search(query_embedding, top_k)
    
    # Объединение найденных документов в контекст
    retrieved_docs = "\n".join([documents[i] for i in I[0]])
    prompt = f"{retrieved_docs}\n\nВопрос: {query}\nОтвет:"
    
    # Генерация ответа
    response = model.generate(model="bambucha/saiga-llama3", prompt=prompt)
    return response['response']

if __name__ == "__main__":
    print("RAG LLM Агент с OLLAMA запущен. Напиши 'exit' для выхода.")
    while True:
        user_input = input("Ты: ")
        if user_input.lower() == "exit":
            break
        response = retrieve_and_generate(user_input)
        print(f"Агент: {response}")


RAG LLM Агент с OLLAMA запущен. Напиши 'exit' для выхода.
Агент: Конечно, я могу помочь вам с ответом. Однако, для этого мне нужно больше информации о томе как ты зарабатываешь эту сумму. И что конкретно ты хочешь узнать?
Агент: Два плюс два — четыре.
Агент: Ответ: 4.
Агент: Ответ: 4.
Агент: Четыре!
Агент: Ответ: четыре.
Агент: Содержание документа...
Агент: Другой контент...


KeyboardInterrupt: 

In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

class QuestionGeneratorAgent:
    def __init__(self, model_name="Narrativa/mT5-base-finetuned-tydiQA-question-generation"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def generate_questions(self, text, num_questions=1):
        input_text = "generate question: " + text
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs,
            max_length=100,
            num_return_sequences=num_questions,
            num_beams=num_questions,
            early_stopping=True
        )
        questions = [self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        return questions

# Пример использования
text = """Какие причины, блять, у тебя были, когда я, буквально, слезно воспринимала и воспринимаю нашу проёбанную дружбу?

Когда ты был для меня вторым человеком, после Миши, потому, что с Мишей у меня были отношени
Думаешь меня, тот факт, что вы в Самару прилетели — нихуя не тронул? Если бы ты не решил это вот так вывалить, мне было бы легче, знаешь

А тут ты решил именно засветить, напомнив о ебаных штанах, которые ты не хотел возвращать, почему-то 

Можешь меня чокнутой считать, ебанутой и так далее, я тебе просто как факт, что мне больно

Хорошо провести время в Самаре"""
question_generator = QuestionGeneratorAgent()
questions = question_generator.generate_questions(text)
print(questions)


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


['Как звали мать Михаила Михаи́ла?']


In [46]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from duckduckgo_search import DDGS
from ollama import Client

from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

class AnswerRetrieverAgent:
    def __init__(self, documents, emb_model='all-MiniLM-L6-v2',llm_model="bambucha/saiga-llama3", ip="http://localhost:11434"):
        self.docs = documents
        self.embedder = SentenceTransformer(emb_model, tokenizer_kwargs={'clean_up_tokenization_spaces':True})
        doc_embs = self.embedder.encode(documents)
        # Создание и добавление в индекс
        self.index = faiss.IndexFlatL2(doc_embs.shape[1])
        self.index.add(np.array(doc_embs))
        self.model= Client(
            host=ip
        )
        self.llm_model_name = llm_model
        
    def retrieve_answer(self, question, top_k=3):
        # Поиск релевантных документов
        query_embedding = self.embedder.encode([question])
        _, I = self.index.search(query_embedding, top_k)
        
        # Объединение найденных документов в контекст
        retrieved_docs = "\n".join([self.docs[i] for i in I[0]])
        prompt = f"{retrieved_docs}\n\nВопрос: {question}\nОтвет:"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return response['response']

    def duckduckgo_search(self, question):
        # Поиск первых 5 результатов по запросу
        results = DDGS().text(question, region='wt-wt', safesearch='Moderate', time='y', max_results=5)
        # Извлечение текстовых фрагментов из результатов
        search_results = [result['snippet'] for result in results if 'snippet' in result]
        prompt = f"{search_results}\n\nВопрос: {question}\nОтвет:"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return response['response']
    



a = AnswerRetrieverAgent(['У тебя три ноги, у тебя точно три ногиб у тебя три ноги и у меня три ноги'])

a.retrieve_answer('сколько ног у тебя')


'К сожалению, ответа на этот вопрос нет. Это не лингвистическая диалоговое задача или математический тест, но rather феномен, который вызывает бесконечную регрессию и логическую дилетанцию.\nУ меня три ноги, я точно знаю\nА у тебя же три ноги, я точно знаю\nНо что это такое? Это означает, что ты - не ты, а ты - то, что есть на твоей ноге?\nИли это значит, что мы все существа с 3-ю конечностью?\nКто-то может сказать: "Я могу проверить", но я отвечу тебе: "Согласно правилам нашего языка, в этом вопросе нет ответа".\nТеперь ты не веришь мне? Я спрашиваю тебя: "Сколько ног у тебя?" Ты не понимаешь, что это означает?\n\nВсё это означает только одно - мы оказались на нездоровой территории. Вот где-то в глубине нашего языка, где даже самые элементарные понятия теряют смысл и превращаются в безумие...'

In [109]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from duckduckgo_search import DDGS
from ollama import Client

from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


class AnswerRetrieverAgent:
    def __init__(self, documents, emb_model='all-MiniLM-L6-v2',llm_model="bambucha/saiga-llama3", ip="http://localhost:11434"):
        self.docs = documents
        self.embedder = SentenceTransformer(emb_model, tokenizer_kwargs={'clean_up_tokenization_spaces':True})
        doc_embs = self.embedder.encode(documents)
        # Создание и добавление в индекс
        self.index = faiss.IndexFlatL2(doc_embs.shape[1])
        self.index.add(np.array(doc_embs))
        self.model= Client(
            host=ip
        )
        self.llm_model_name = llm_model
        
    def retrieve_answer(self, question, top_k=3):
        # Поиск релевантных документов
        query_embedding = self.embedder.encode([question])
        _, I = self.index.search(query_embedding, top_k)
        
        # Объединение найденных документов в контекст
        retrieved_docs = "\n".join([self.docs[i] for i in I[0]])
        prompt = f"{retrieved_docs}\n\nОтвечай коротко и по делу на русском языке. Вопрос: {question}\nОтвет:"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return response['response']

    def duckduckgo_search(self, question):
        # Поиск первых 5 результатов по запросу
        results = DDGS().text(question, region='wt-wt', safesearch='Moderate', timelimit='y', max_results=5)
        # Извлечение текстовых фрагментов из результатов
        search_results = [result['snippet'] for result in results if 'snippet' in result]
        prompt = f"{search_results}\n\nОтвечай коротко и по делу на русском языке. Вопрос: {question}\nОтвет:"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return response['response']
    


class QuestionGeneratorAgent:
    def __init__(self, model_name="Narrativa/mT5-base-finetuned-tydiQA-question-generation"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def generate_questions(self, text, num_questions=1):
        input_text = "Сгенерируй вопрос, на тематику киберспорта: " + text
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs,
            max_length=100,
            num_return_sequences=num_questions,
            num_beams=num_questions,
            early_stopping=True
        )
        questions = [self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        return questions

class LLMQuestionAnsweringAgent:
    def __init__(self, docs, question_model_name="Narrativa/mT5-base-finetuned-tydiQA-question-generation", emb_model='all-MiniLM-L6-v2',llm_model="bambucha/saiga-llama3", rag_ip="http://localhost:11434"):
        self.question_agent = QuestionGeneratorAgent(model_name=question_model_name)
        self.answer_agent = AnswerRetrieverAgent(docs, emb_model=emb_model,llm_model=llm_model, ip=rag_ip)

    def process_text(self, text):
        questions = self.question_agent.generate_questions(text)
        qa_pairs = []
        for question in questions:
            answer = self.answer_agent.retrieve_answer(question)
            web_results = self.answer_agent.duckduckgo_search(question)
            qa_pairs.append({
                "question": question,
                "model_answer": answer,
                "web_results": web_results
            })
        return qa_pairs
    def process_text(self, texts):
        questions = self.question_agent.generate_questions(texts)
        qa_pairs = []
        for question in questions:
            answer = self.answer_agent.retrieve_answer(question)
            web_results = self.answer_agent.duckduckgo_search(question)
            qa_pairs.append({
                "question": question,
                "model_answer": answer,
                "web_results": web_results
            })
        return qa_pairs

In [ ]:
qa_dataset = []

In [111]:
import random
random.seed()
docs_for_questions = random.sample(raw_documents['text'].to_list(), 100)
agent = LLMQuestionAnsweringAgent(docs=raw_documents['text'])
for doc in docs_for_questions:
    temp = agent.process_text(doc)[0]
    print(f"Вопрос: {temp['question']}")
    print(f"Ответ модели: {temp['model_answer']}")
    print(f"Результаты поиска: {temp['web_results']}")
    print()
    qa_dataset.append(temp)


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Вопрос: Какой игрок является разработчиком игры «Си́н»?
Ответ модели: Яндекс.
Результаты поиска: Алексей Зиновкин.

Вопрос: Какой игровой контент на 2019 год?
Ответ модели: К сожалению, в данный момент информация о поддерживаемом контенте от Меррилла в 2019 году не доступна. Но можно добавить что в последние годы компания разрабатывает и выпускает новые игры и обновления к уже существующим, а также предоставляет различные услуги для геймеров.
Результаты поиска: Игровой контент на 2019 год: Destiny 2, Apex Legends, Overwatch, Fortnite, PUBG, World of Warcraft, Guild Wars 2 и многое другое.

Вопрос: Как звали Марка Мартина впервые?
Ответ модели: Марк, как обычно.
Результаты поиска: Марсель Депасе.

Вопрос: Как звали мать Бонасера?
Ответ модели: Вито.
Результаты поиска: Маргарита.

Вопрос: Какая была самая большая игра в мире?
Ответ модели: Самой большой игры в мире я не знаю, но уверен, что каждый человек имеет право выбирать свою игру.
Результаты поиска: Супер Боул.

Вопрос: Какая верси

KeyboardInterrupt: 

In [107]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from duckduckgo_search import DDGS
from ollama import Client

from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


class AnswerRetrieverAgent:
    def __init__(self, documents, emb_model='all-MiniLM-L6-v2',llm_model="bambucha/saiga-llama3", ip="http://localhost:11434"):
        self.docs = documents
        self.embedder = SentenceTransformer(emb_model, tokenizer_kwargs={'clean_up_tokenization_spaces':True})
        doc_embs = self.embedder.encode(documents)
        # Создание и добавление в индекс
        self.index = faiss.IndexFlatL2(doc_embs.shape[1])
        self.index.add(np.array(doc_embs))
        self.model= Client(
            host=ip
        )
        self.llm_model_name = llm_model
        
    def retrieve_answer(self, question, top_k=3):
        # Поиск релевантных документов
        query_embedding = self.embedder.encode([question])
        _, I = self.index.search(query_embedding, top_k)
        
        # Объединение найденных документов в контекст
        retrieved_docs = "\n".join([self.docs[i] for i in I[0]])
        prompt = f"{retrieved_docs}\n\nОтвечай коротко и по делу на русском языке. Вопрос: {question}\nОтвет:"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return response['response']

    def duckduckgo_search(self, question):
        # Поиск первых 5 результатов по запросу
        results = DDGS().text(question, region='wt-wt', safesearch='Moderate', timelimit='y', max_results=5)
        # Извлечение текстовых фрагментов из результатов
        search_results = [result['snippet'] for result in results if 'snippet' in result]
        prompt = f"{search_results}\n\nОтвечай коротко и по делу на русском языке. Вопрос: {question}\nОтвет:"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return response['response']
    


class QuestionGeneratorAgent:
    def __init__(self, llm_model="bambucha/saiga-llama3", ip="http://localhost:11434"):
        self.model= Client(
            host=ip
        )
        self.llm_model_name = llm_model
    def generate_questions(self, text, num_questions=1):
        prompt = f"Говори коротко и по делу на русском языке. Придумай {num_questions} вопрос/ов на тему игр и киберспорта, основанный на тексте {text}"
        
        # Генерация ответа
        response = self.model.generate(model=self.llm_model_name, prompt=prompt)
        return [response['response']]

class LLMQuestionAnsweringAgent:
    def __init__(self, docs, emb_model='all-MiniLM-L6-v2',llm_model="bambucha/saiga-llama3", rag_ip="http://localhost:11434"):
        self.question_agent = QuestionGeneratorAgent(llm_model=llm_model, ip=rag_ip)
        self.answer_agent = AnswerRetrieverAgent(docs, emb_model=emb_model,llm_model=llm_model, ip=rag_ip)

    def process_text(self, text):
        qa_pairs = []
        try:
            questions = self.question_agent.generate_questions(text)
            for question in questions:
                answer = self.answer_agent.retrieve_answer(question)
                web_results = self.answer_agent.duckduckgo_search(question)
                qa_pairs.append({
                    "question": question,
                    "model_answer": answer,
                    "web_results": web_results
                })
        except Exception:
            pass
        return qa_pairs

In [108]:
import random
docs_for_questions = random.sample(raw_documents['text'].to_list(), 100)

agent = LLMQuestionAnsweringAgent(docs=raw_documents['text'])
for doc in docs_for_questions:
    try:
        temp = agent.process_text(doc)[0]
        print(f"Вопрос: {temp['question']}")
        print(f"Ответ модели: {temp['model_answer']}")
        print(f"Результаты поиска: {temp['web_results']}")
        print()
        qa_dataset.append(temp)
    except Exception:
        pass

Вопрос: Привет! Какой из игровых жанров ты считаешь самым популярным в последнее время? 

(Ответ: На этой основе можно задать вопросы о популярных играх, тенденциях в киберспорте, анализе трендов и предпочтениях пользователей.)
Ответ модели: Привет! Согласно данным и наблюдениям, на данный момент самым популярным жанром является battle royale.
Результаты поиска: Браузерные FPS - это текущий лидер по популярности.

Вопрос: Сайга! Как ты думаешь, ключевое значение для успеха в киберспорте имеет боевая или физическая профессиональная подготовка?
Ответ модели: Нет, я думаю, что ключевое значение имеет умение анализировать ситуации и принимать правильные решения на основе полученной информации. Боевую подготовку можно заменить стратегическим мышлением и умением управлять командой и индивидуумами на поле. Это позволит игрокам более успешно действовать в киберспорте, так как в нем играет большая роль умение анализировать ситуации и принимать правильные решения.
Результаты поиска: Боевая подго

KeyboardInterrupt: 

In [112]:
len(qa_dataset)

111

In [103]:
pd.DataFrame(qa_dataset).to_excel('dataset.xlsx')
